In [ ]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time

# データベースの名前
db_name = 'google_repos.db'

# 1. データベースの準備
# まずデータベースに接続する（なければ作られる）
conn = sqlite3.connect(db_name)
cur = conn.cursor()

# 毎回やり直せるように、古いテーブルがあったら消す
cur.execute('DROP TABLE IF EXISTS repositories')

# 新しくテーブルを作る
cur.execute('''
    CREATE TABLE repositories (
        name TEXT,
        language TEXT,
        stars TEXT
    )
''')

# 変更を確定する
conn.commit()


# 2. スクレイピングの準備
url = 'https://github.com/orgs/google/repositories'

# Google Chromeのふりをするための設定（これをしないとブロックされることがあるらしい）
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

print("スクレイピングを開始します...")

# サイトの情報を取得する
response = requests.get(url, headers=headers)

# ちゃんと取得できたか確認（200ならOK）
if response.status_code == 200:
    # HTMLを解析できる形にする
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # リポジトリのリスト（liタグ）を全部持ってくる
    # class名は検証ツールで調べたもの
    repo_list = soup.find_all('li', class_='Box-row')
    
    print(f"{len(repo_list)} 件のデータが見つかりました。")

    # ひとつずつ取り出して処理する
    for repo in repo_list:
        
        # --- リポジトリ名を取得 ---
        # itempro="name codeRepository" という属性を持つ aタグを探す
        name_tag = repo.find('a', itemprop='name codeRepository')
        if name_tag:
            # タグの中の文字を取り出して、余計な空白を消す
            repo_name = name_tag.text.strip()
        else:
            repo_name = "名前なし"

        # --- 言語を取得 ---
        # itemprop="programmingLanguage" という属性を持つ spanタグを探す
        lang_tag = repo.find('span', itemprop='programmingLanguage')
        if lang_tag:
            lang_name = lang_tag.text.strip()
        else:
            lang_name = "言語なし"

        # --- スター数を取得 ---
        # スター数は特定のリンク（hrefに stargazers が入っているもの）の中にある
        # 正規表現は難しいので、aタグを全部見て探す方法にする
        star_num = "0"
        all_links = repo.find_all('a')
        
        for link in all_links:
            # href属性があるか確認してから、中に 'stargazers' が含まれているかチェック
            if link.get('href') and 'stargazers' in link.get('href'):
                star_num = link.text.strip()
                # カンマが入ってると数値として扱いづらいので消しておく
                star_num = star_num.replace(',', '')
                break # 見つかったらループを抜ける

        
        # 画面に表示してみる
        print(f"取得中: {repo_name} / {lang_name} / {star_num}")

        # データベースに追加する
        cur.execute('INSERT INTO repositories (name, language, stars) VALUES (?, ?, ?)', (repo_name, lang_name, star_num))
        
        # 課題の指定通り、1秒待つ
        time.sleep(1)

    # 最後にまとめて保存する
    conn.commit()
    print("保存が完了しました。")

else:
    print("サイトにアクセスできませんでした。")


# 3. データの確認
print("\n--- データベースの中身を表示します ---")

# 全部のデータを取ってくる
cur.execute('SELECT * FROM repositories')
data_list = cur.fetchall()

for data in data_list:
    print(data)

# データベースを閉じる
conn.close()